## Prior based automatic segmentation of deep nuclei

In this notebook a method for automatic segmentation on older patients deep nuclei using a prior from younger ones is described.

For each target nuclei, the **center of gravity** of the prior mask is computed. A box is constructed with a specific **side** (usually 10-12 voxel) centered on this value, in order to have a **local evaluation**

Coordinates of this box, together with scaled image gray values are the features that we're intereset in.

A KMeans algorithm is run with **n_cluster** (default=10) in order to find consistent region of images. 

The algorithm choose randomly **n_cluster** centroids and compute the euclidian distance of each point to the centroids, weighted with the probability of belonging to the right cluster. The objective is to minimize this cost function

${\displaystyle J(W,C)=\sum _{i=1}^{n}\sum _{j=1}^{c}w_{ij}^{m}\left\|\mathbf {x} _{i}-\mathbf {c} _{j}\right\|^{2}}$

${\displaystyle w_{ij}={\frac {1}{\sum _{k=1}^{c}\left({\frac {\left\|\mathbf {x} _{i}-\mathbf {c} _{j}\right\|}{\left\|\mathbf {x} _{i}-\mathbf {c} _{k}\right\|}}\right)^{\frac {2}{m-1}}}}}$

Each point will have **n_cluster** distances from each one of the centroids and it is associated the cluster of the closest centroid.
Now, for each cluster, the new centroids are computed as the mean coordinates of the points

${\displaystyle c_{k}={{\sum _{x}{w_{k}(x)}^{m}x} \over {\sum _{x}{w_{k}(x)}^{m}}},}$


and the process is repeated until convergence

When performing predictions, each point of the local box is associated with a probability of belonging to each cluster.
The cluster that contains the center of mass is selected as the target one, and probability of this cluster is increased by $\alpha P_{prior}$, then the argmax is taken to provide final mask.

A connected component analysis in 3D looking for 26 nearest neighbors is conducted to keep the largest connected components and avoid fragmented masks. The final mask is a boolean 3D box.

The box is placed in a 3D zeros array with the same shape of the original image


In [18]:
from segmentation import *
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from os.path import join as opj
import tqdm
import pandas as pd
from nilearn.image import mean_img

In [2]:
prior_path="priors_12younger_inMIITRAv2space"
data_path="data_15elder_inMIITRAv2space"

target_dir="fcm_segmentations"

In [3]:
priors=glob.glob(opj(prior_path,"*.nii.gz"))
data=glob.glob(opj(data_path,"*.nii.gz"))

In [4]:
img_nifti=nib.load(data[0])
img=img_nifti.get_fdata()
prior=nib.load(priors[0]).get_fdata()

In [5]:
for i,d in enumerate(data):
    
    d_name=d.split("/")[-1].split(".nii.gz")[0]
    
    print(f"Performing segmentation for {d_name} ({i}/{len(data)}")
    os.makedirs(opj(target_dir,d_name),exist_ok=True)
    img_nifti=nib.load(d)
    affine=img_nifti.affine
    img=img_nifti.get_fdata()
    for p in tqdm.tqdm(priors):
        
        p_name=p.split("/")[-1]

        prior=nib.load(p).get_fdata()
        seg=run_prior_segmentation(img,prior,n_cluster=10,half_side=6,prior_weight=0.5)
        out=nib.Nifti1Image(seg,affine=affine)
        nib.save(out,opj(target_dir,d_name,p_name))

Performing segmentation for dtifit_d_S0_bfc_sub8_masked_2_MIITRA2 (0/15


100%|█████████████████████████████████████████| 120/120 [01:49<00:00,  1.10it/s]


Performing segmentation for dtifit_d_S0_bfc_sub18_masked_2_MIITRA2 (1/15


100%|█████████████████████████████████████████| 120/120 [01:48<00:00,  1.11it/s]


Performing segmentation for dtifit_d_S0_bfc_sub17_masked_2_MIITRA2 (2/15


...

100%|█████████████████████████████████████████| 120/120 [01:47<00:00,  1.12it/s]


Performing segmentation for dtifit_d_S0_bfc_sub24_masked_2_MIITRA2 (13/15


100%|█████████████████████████████████████████| 120/120 [01:47<00:00,  1.12it/s]


Performing segmentation for dtifit_d_S0_bfc_sub13_masked_2_MIITRA2 (14/15


100%|█████████████████████████████████████████| 120/120 [01:47<00:00,  1.12it/s]


In [6]:
def dice(im1, im2):
    """
    Computes the Dice coefficient, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    dice : float
        Dice coefficient as a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
        
    Notes
    -----
    The order of inputs for `dice` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / (im1.sum() + im2.sum())## Evaluation

In [7]:
prior_names=os.listdir(prior_path)
d_name=d.split("/")[-1].split(".nii.gz")[0]

dice_dict={}
for p in tqdm.tqdm(prior_names):
    
    dice_dict[p]=[]
    
    for i,d in enumerate(data):
    
        d_name=d.split("/")[-1].split(".nii.gz")[0]
        ref_file=opj(prior_path,p)
        target_file=opj(target_dir,d_name,p)
        try:
            dice_score=dice(nib.load(target_file).get_fdata(),nib.load(ref_file).get_fdata())
            dice_dict[p].append(dice_score)
        except:
            print(f"file {target_file} or {ref_file} is not a nifti")
    
    # opj("km_segmentations",d_name

  0%|                                                   | 0/121 [00:00<?, ?it/s]/tmp/ipykernel_2070059/2134065819.py:22: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  im1 = np.asarray(im1).astype(np.bool)
/tmp/ipykernel_2070059/2134065819.py:23: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  im2 = np.asarray(im2).astype(np.bool)
 16%|██████▌        

file fcm_segmentations/dtifit_d_S0_bfc_sub8_masked_2_MIITRA2/zList_nucleilabels_name.txt or priors_12younger_inMIITRAv2space/zList_nucleilabels_name.txt is not a nifti
file fcm_segmentations/dtifit_d_S0_bfc_sub18_masked_2_MIITRA2/zList_nucleilabels_name.txt or priors_12younger_inMIITRAv2space/zList_nucleilabels_name.txt is not a nifti
file fcm_segmentations/dtifit_d_S0_bfc_sub17_masked_2_MIITRA2/zList_nucleilabels_name.txt or priors_12younger_inMIITRAv2space/zList_nucleilabels_name.txt is not a nifti
file fcm_segmentations/dtifit_d_S0_bfc_sub3_masked_2_MIITRA2/zList_nucleilabels_name.txt or priors_12younger_inMIITRAv2space/zList_nucleilabels_name.txt is not a nifti
file fcm_segmentations/dtifit_d_S0_bfc_sub23_masked_2_MIITRA2/zList_nucleilabels_name.txt or priors_12younger_inMIITRAv2space/zList_nucleilabels_name.txt is not a nifti
file fcm_segmentations/dtifit_d_S0_bfc_sub7_masked_2_MIITRA2/zList_nucleilabels_name.txt or priors_12younger_inMIITRAv2space/zList_nucleilabels_name.txt is n

100%|█████████████████████████████████████████| 121/121 [08:56<00:00,  4.43s/it]


In [10]:
dice_dict={i:v for i,v in dice_dict.items() if len(v)}
df=pd.DataFrame.from_dict(dice_dict)
df

,s5_l10_2_MIITRA2.nii.gz,s5_r10_2_MIITRA2.nii.gz,s7_r10_2_MIITRA2.nii.gz,s4_r2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s12_l3_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s12_r2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s2_l2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s11_l10_2_MIITRA2.nii.gz,s1_r3_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s5_r2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,...,s10_l2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s12_l9_2_MIITRA2.nii.gz,s9_l1_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s8_r1_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s1_r2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s10_r10_2_MIITRA2.nii.gz,s2_l3_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s1_l1_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s7_r2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s8_l1_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz
0,0.227907,0.270096,0.188976,0.138408,0.291613,0.238806,0.238806,0.301158,0.320356,0.082474,...,0.167742,0.289474,0.355000,0.238908,0.173302,0.247863,0.263033,0.374753,0.169972,0.286885
1,0.120000,0.362069,0.210526,0.236715,0.253416,0.336842,0.268537,0.136821,0.475843,0.179012,...,0.369231,0.425121,0.421875,0.000000,0.378378,0.102273,0.228942,0.345238,0.307179,0.303279
2,0.324324,0.098837,0.121581,0.289216,0.391753,0.290672,0.295082,0.261682,0.476364,0.225705,...,0.398190,0.349398,0.459530,0.274336,0.409222,0.078049,0.371459,0.453152,0.341564,0.359322
3,0.295918,0.117400,0.132505,0.230576,0.463646,0.238411,0.335484,0.275618,0.374343,0.155039,...,0.341463,0.335025,0.455090,0.288066,0.255670,0.153005,0.279736,0.361266,0.266667,0.283582
4,0.116129,0.505263,0.341969,0.236422,0.348675,0.340580,0.344615,0.332075,0.414508,0.157205,...,0.314050,0.295775,0.008000,0.000000,0.270742,0.240602,0.281352,0.435754,0.311224,0.000000
5,0.181818,0.205788,0.102740,0.236776,0.328798,0.325333,0.345572,0.208696,0.382609,0.205882,...,0.419643,0.177340,0.214634,0.220994,0.333333,0.082418,0.280040,0.447712,0.297189,0.000000
6,0.095012,0.256055,0.094828,0.244604,0.482921,0.291777,0.285714,0.156863,0.493333,0.192308,...,0.291480,0.318681,0.445087,0.256410,0.303226,0.062696,0.278876,0.332665,0.297297,0.362140
7,0.344828,0.240310,0.081911,0.167421,0.292108,0.271357,0.302158,0.321951,0.314402,0.150943,...,0.337950,0.295918,0.342342,0.328767,0.339250,0.137097,0.289760,0.565957,0.229787,0.000000
8,0.234432,0.091398,0.097959,0.149837,0.272512,0.177914,0.270023,0.145946,0.330761,0.151030,...,0.310680,0.261224,0.527174,0.000000,0.213178,0.066667,0.253763,0.457565,0.253333,0.388664
9,0.120192,0.085561,0.000000,0.225895,0.399507,0.283525,0.230384,0.123007,0.408714,0.178218,...,0.300654,0.286822,0.464986,0.000000,0.309735,0.088889,0.351381,0.410749,0.213483,0.339768


In [11]:
df.to_csv("fcm_segmentations.csv",index=True)

In [13]:
df.describe()

,s5_l10_2_MIITRA2.nii.gz,s5_r10_2_MIITRA2.nii.gz,s7_r10_2_MIITRA2.nii.gz,s4_r2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s12_l3_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s12_r2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s2_l2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s11_l10_2_MIITRA2.nii.gz,s1_r3_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s5_r2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,...,s10_l2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s12_l9_2_MIITRA2.nii.gz,s9_l1_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s8_r1_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s1_r2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s10_r10_2_MIITRA2.nii.gz,s2_l3_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s1_l1_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s7_r2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s8_l1_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz
count,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,...,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
mean,0.243330,0.225677,0.136637,0.205178,0.366268,0.276163,0.289634,0.257628,0.376468,0.166478,...,0.323728,0.342522,0.285068,0.138136,0.298643,0.114142,0.287734,0.386579,0.278287,0.178704
std,0.107057,0.116451,0.078828,0.051265,0.074751,0.053277,0.043190,0.094267,0.084942,0.039664,...,0.069282,0.091744,0.206803,0.136201,0.073926,0.059850,0.036343,0.124885,0.064197,0.175219
min,0.095012,0.085561,0.000000,0.121827,0.253416,0.177914,0.228029,0.123007,0.152231,0.082474,...,0.167742,0.177340,0.000000,0.000000,0.168627,0.040816,0.228942,0.000000,0.163265,0.000000
25%,0.151005,0.116077,0.096393,0.158629,0.310453,0.238608,0.253672,0.161138,0.327160,0.150987,...,0.296067,0.292624,0.055813,0.000000,0.263206,0.080233,0.273141,0.353252,0.241560,0.000000
50%,0.234432,0.240310,0.121581,0.225895,0.348675,0.290672,0.295082,0.275618,0.377079,0.158228,...,0.314050,0.318681,0.355000,0.220994,0.305785,0.102273,0.279736,0.410749,0.297189,0.283582
75%,0.315734,0.273108,0.178362,0.236745,0.427416,0.322087,0.327434,0.328717,0.421903,0.185660,...,0.355347,0.387259,0.457310,0.247659,0.350113,0.124259,0.290354,0.450432,0.312285,0.348346
max,0.425121,0.505263,0.341969,0.289216,0.482921,0.340580,0.345572,0.420290,0.493333,0.234483,...,0.450575,0.520833,0.527174,0.328767,0.409222,0.247863,0.371459,0.565957,0.389497,0.388664


In [14]:
(df==0).sum().sum()

103

## Make them probalistic out

In [16]:
target_names=["s*_l10_2_MIITRA2.nii.gz",                 
"s*_l1_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz", 
"s*_l2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz", 
"s*_l3_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz",
"s*_l9_2_MIITRA2.nii.gz", 
"s*_r10_2_MIITRA2.nii.gz",                 
"s*_r1_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz", 
"s*_r2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz", 
"s*_r3_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz",
"s*_r9_2_MIITRA2.nii.gz"]


In [28]:
for sub in tqdm.tqdm(os.listdir("fcm_segmentations")):
    sub_dir=opj("fcm_segmentations",sub)
    out_dir=opj("fcm_segmentations",sub,"final")
    os.makedirs(out_dir, exist_ok=True)
    os.makedirs(opj(out_dir,"probabilistic"),exist_ok=True)
    os.makedirs(opj(out_dir,"binary_thr_0"),exist_ok=True)
    os.makedirs(opj(out_dir,"binary_thr_25"),exist_ok=True)
    os.makedirs(opj(out_dir,"binary_thr_50"),exist_ok=True)
    os.makedirs(opj(out_dir,"binary_thr_75"),exist_ok=True)
    
    for t in target_names:
        segmentations=[]
        for i in range(1,13):
            affine=None
            try:
                segmentations.append(nib.load(opj(sub_dir,t.replace("*",f"{i}"))))
            except:
                print(f"could not load {opj(sub_dir,t.replace('*',f'{i}'))}")
        
        if len(segmentations):
            prob_segmentation=mean_img(segmentations)
            data=prob_segmentation.get_fdata()

            #probabilistic map
            data=np.clip(data,0,1)

            prob_nib=nib.Nifti1Image(data,prob_segmentation.affine)
            nib.save(prob_nib,opj(out_dir,"probabilistic",t))

            #boolean map

            data_bool=(data>0)*1
            bool_nib=nib.Nifti1Image(data_bool,prob_segmentation.affine)
            nib.save(bool_nib,opj(out_dir,"binary_thr_0",t))

            #threshold at 0.25

            data_thr=(data>0.25)*1
            thr_nib=nib.Nifti1Image(data_thr,prob_segmentation.affine)
            nib.save(thr_nib,opj(out_dir,"binary_thr_25",t))

            #threshold at 0.50

            data_thr=(data>0.50)*1
            thr_nib=nib.Nifti1Image(data_thr,prob_segmentation.affine)
            nib.save(thr_nib,opj(out_dir,"binary_thr_50",t))

            #threshold at 0.75

            data_thr=(data>0.75)*1
            thr_nib=nib.Nifti1Image(data_thr,prob_segmentation.affine)
            nib.save(thr_nib,opj(out_dir,"binary_thr_75",t))


 56%|████████████████████████▊                   | 9/16 [06:04<04:44, 40.62s/it]

could not load fcm_segmentations/.ipynb_checkpoints/s1_l10_2_MIITRA2.nii.gz
could not load fcm_segmentations/.ipynb_checkpoints/s2_l10_2_MIITRA2.nii.gz
could not load fcm_segmentations/.ipynb_checkpoints/s3_l10_2_MIITRA2.nii.gz
could not load fcm_segmentations/.ipynb_checkpoints/s4_l10_2_MIITRA2.nii.gz
could not load fcm_segmentations/.ipynb_checkpoints/s5_l10_2_MIITRA2.nii.gz
could not load fcm_segmentations/.ipynb_checkpoints/s6_l10_2_MIITRA2.nii.gz
could not load fcm_segmentations/.ipynb_checkpoints/s7_l10_2_MIITRA2.nii.gz
could not load fcm_segmentations/.ipynb_checkpoints/s8_l10_2_MIITRA2.nii.gz
could not load fcm_segmentations/.ipynb_checkpoints/s9_l10_2_MIITRA2.nii.gz
could not load fcm_segmentations/.ipynb_checkpoints/s10_l10_2_MIITRA2.nii.gz
could not load fcm_segmentations/.ipynb_checkpoints/s11_l10_2_MIITRA2.nii.gz
could not load fcm_segmentations/.ipynb_checkpoints/s12_l10_2_MIITRA2.nii.gz
could not load fcm_segmentations/.ipynb_checkpoints/s1_l1_Seg_4cl_S0_WBednoCSF_2_MIIT

100%|███████████████████████████████████████████| 16/16 [10:08<00:00, 38.05s/it]


## Final evaluations

In [30]:
prior_path
os.makedirs("average_prior",exist_ok=True)

In [68]:
avg_img=[]

os.makedirs(opj("average_prior","probabilistic"),exist_ok=True)
os.makedirs(opj("average_prior","binary_thr_0"),exist_ok=True)
os.makedirs(opj("average_prior","binary_thr_25"),exist_ok=True)
os.makedirs(opj("average_prior","binary_thr_50"),exist_ok=True)
os.makedirs(opj("average_prior","binary_thr_75"),exist_ok=True)

for t in tqdm.tqdm(target_names):
    priors=[]
    affine=None
    for i in range(1,13):
        
        try:
            
            priors.append(nib.load(opj(prior_path,t.replace("*",f"{i}"))))
        except:
            print(f"could not load {opj(sub_dir,t.replace('*',f'{i}'))}")


    prob_segmentation=mean_img(priors)
    data=prob_segmentation.get_fdata()
    

    #probabilistic map
    data=np.clip(data,0,1)

    prob_nib=nib.Nifti1Image(data,prob_segmentation.affine)
    nib.save(prob_nib,opj("average_prior","probabilistic",t))

    #boolean map

    data_bool=(data>0)*1
    bool_nib=nib.Nifti1Image(data_bool,prob_segmentation.affine)
    nib.save(bool_nib,opj("average_prior","binary_thr_0",t))

    #threshold at 0.25

    data_thr=(data>0.25)*1
    thr_nib=nib.Nifti1Image(data_thr,prob_segmentation.affine)
    nib.save(thr_nib,opj("average_prior","binary_thr_25",t))

    #threshold at 0.50

    data_thr=(data>0.50)*1
    thr_nib=nib.Nifti1Image(data_thr,prob_segmentation.affine)
    nib.save(thr_nib,opj("average_prior","binary_thr_50",t))

    #threshold at 0.75

    data_thr=(data>0.75)*1
    thr_nib=nib.Nifti1Image(data_thr,prob_segmentation.affine)
    nib.save(thr_nib,opj("average_prior","binary_thr_75",t))


100%|███████████████████████████████████████████| 10/10 [00:56<00:00,  5.62s/it]


## For each patient, load all the final images and compute the dice for each nucleus

In [ ]:
def compute_final_dice(modality):
    dice_dict={}

    for t in tqdm.tqdm(target_names):
        dice_dict[t]=[]
        for sub in os.listdir("fcm_segmentations"):
            sub_dir=opj("fcm_segmentations",sub,"final",modality)

            try:
                avg_prior=nib.load(opj("average_prior",modality,t))
                sub_seg = nib.load(opj(sub_dir,t))

                try:
                    dice_score=dice(avg_prior.get_fdata(),sub_seg.get_fdata())

                    dice_dict[t].append(dice_score)
                except Exception as e:
                    # print(e)
                    print(f"file {opj(sub_dir,t)} or {opj('average_prior',modality,t)} is not a nifti")
            except:
                print(f"cant load file from {opj(sub_dir,t)}")

    dice_dict={i:v for i,v in dice_dict.items() if len(v)}
    df=pd.DataFrame.from_dict(dice_dict)
    sub_list=[i for i in os.listdir("fcm_segmentations") if "MIITRA" in i]
    df["sub"]=sub_list
    columns=["sub"]+target_names
    df=df[columns]
    
    df.to_csv(f"dice_average_prior_{modality}.csv")
    return df

In [83]:

modality="binary_thr_25"

for modality in ["binary_thr_0","binary_thr_25","binary_thr_50","binary_thr_75"]:
    compute_final_dice(modality)

  0%|                                                    | 0/10 [00:00<?, ?it/s]/tmp/ipykernel_2070059/2134065819.py:22: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  im1 = np.asarray(im1).astype(np.bool)
/tmp/ipykernel_2070059/2134065819.py:23: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  im2 = np.asarray(im2).astype(np.bool)


cant load file from fcm_segmentations/.ipynb_checkpoints/final/binary_thr_0/s*_l10_2_MIITRA2.nii.gz


 10%|████▍                                       | 1/10 [00:04<00:41,  4.61s/it]

cant load file from fcm_segmentations/.ipynb_checkpoints/final/binary_thr_0/s*_l1_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz


 20%|████████▊                                   | 2/10 [00:08<00:33,  4.13s/it]

...

 80%|███████████████████████████████████▏        | 8/10 [00:34<00:08,  4.29s/it]

cant load file from fcm_segmentations/.ipynb_checkpoints/final/binary_thr_75/s*_r3_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz


 90%|███████████████████████████████████████▌    | 9/10 [00:38<00:04,  4.28s/it]

cant load file from fcm_segmentations/.ipynb_checkpoints/final/binary_thr_75/s*_r9_2_MIITRA2.nii.gz


100%|███████████████████████████████████████████| 10/10 [00:42<00:00,  4.29s/it]


In [78]:
dice_dict={i:v for i,v in dice_dict.items() if len(v)}
df=pd.DataFrame.from_dict(dice_dict)
sub_list=[i for i in os.listdir("fcm_segmentations") if "MIITRA" in i]
df["sub"]=sub_list
columns=["sub"]+target_names
df[columns]

,sub,s*_l10_2_MIITRA2.nii.gz,s*_l1_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s*_l2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s*_l3_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s*_l9_2_MIITRA2.nii.gz,s*_r10_2_MIITRA2.nii.gz,s*_r1_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s*_r2_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s*_r3_Seg_4cl_S0_WBednoCSF_2_MIITRA2.nii.gz,s*_r9_2_MIITRA2.nii.gz
0,dtifit_d_S0_bfc_sub8_masked_2_MIITRA2,0.240786,0.403326,0.324324,0.379909,0.311864,0.255556,0.301587,0.193353,0.406147,0.304688
1,dtifit_d_S0_bfc_sub6_masked_2_MIITRA2,0.081633,0.390135,0.394161,0.491968,0.331797,0.167619,0.418803,0.304511,0.510638,0.116022
2,dtifit_d_S0_bfc_sub19_masked_2_MIITRA2,0.314894,0.665025,0.348276,0.327442,0.287037,0.172043,0.698225,0.234513,0.307692,0.158996
3,dtifit_d_S0_bfc_sub17_masked_2_MIITRA2,0.158491,0.443564,0.366038,0.414327,0.371859,0.131687,0.377551,0.314908,0.495337,0.207650
4,dtifit_d_S0_bfc_sub11_masked_2_MIITRA2,0.186916,0.462151,0.290837,0.418395,0.458333,0.153086,0.479532,0.203267,0.401487,0.116992
5,dtifit_d_S0_bfc_sub1_masked_2_MIITRA2,0.286738,0.289100,0.395402,0.393476,0.361345,0.159011,0.000000,0.354379,0.471111,0.318966
6,dtifit_d_S0_bfc_sub3_masked_2_MIITRA2,0.280822,0.498208,0.397727,0.468284,0.298932,0.150171,0.377273,0.231183,0.454638,0.128000
7,dtifit_d_S0_bfc_sub7_masked_2_MIITRA2,0.135364,0.490842,0.344086,0.386910,0.208000,0.182278,0.436090,0.277657,0.473490,0.275591
8,dtifit_d_S0_bfc_sub13_masked_2_MIITRA2,0.355932,0.514071,0.432718,0.410839,0.505155,0.231975,0.461538,0.280822,0.484043,0.444444
9,dtifit_d_S0_bfc_sub22_masked_2_MIITRA2,0.127753,0.503145,0.313559,0.399616,0.311258,0.065217,0.437895,0.307292,0.474619,0.207317


In [87]:
## Clean single files

for sub in tqdm.tqdm(os.listdir("fcm_segmentations")):
    to_remove=glob.glob(opj("fcm_segmentations",sub,"*.nii.gz"))
    for i in to_remove:
        os.remove(i)

100%|██████████████████████████████████████████| 16/16 [00:00<00:00, 103.63it/s]
